In [1]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_inx, valid_inx = stratified_sampling(Y)
print("Train {}; Valid {};".format(len(train_inx), len(valid_inx)))

w_size = 192
h_size = 192
input_shape = (w_size, h_size, 3)
print(input_shape)

Using TensorFlow backend.


Import time: 10.21570634841919
Import time: 5.245208740234375e-05
Shape of Y (40479, 17)
Train 36431; Valid 4048;
(192, 192, 3)


In [2]:
start = time()
X = load_images(train_df['image_name'].values, DATA_DIR + '/train-jpg', True, w_size, h_size)
print(X.shape)
print(time() - start)

(40479, 192, 192, 3)
29.05090832710266


In [3]:
from keras.applications.vgg16 import VGG16
vgg16_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)

# vgg16_model.layers.pop() # removing last MaxPooling layer
# vgg16_model = Sequential(vgg16_model.layers) # reinitialize
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 192, 192, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 192, 192, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 192, 192, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 96, 96, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 96, 96, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 96, 96, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 48, 48, 128)       0         
__________

In [4]:
vgg16_train_feat = vgg16_model.predict(X, batch_size=BATCH_SIZE, verbose = 1)
save_array(DATA_DIR + '/models/vgg16_train_feat.dat', vgg16_train_feat)

40479/40479 [==============================] - 496s   


In [5]:
del X
del vgg16_train_feat

start = time()
X_test = load_images(test_df['image_name'].values, DATA_DIR + '/test-jpg', True, w_size, h_size)
print(X_test.shape)
print(time() - start)

(61191, 192, 192, 3)
60.45122933387756


In [6]:
vgg16_test_feat = vgg16_model.predict(X_test, batch_size=BATCH_SIZE, verbose = 1)
save_array(DATA_DIR + '/models/vgg16_test_feat.dat', vgg16_test_feat)

61191/61191 [==============================] - 745s   


In [ ]:
# vgg16_train_feat = load_array(DATA_DIR + '/models/vgg16_train_feat.dat')
# print(vgg16_train_feat.shape)

# vgg16_test_feat = load_array(DATA_DIR + '/models/vgg16_test_feat.dat')
# print(vgg16_test_feat.shape)